### Customer Service Agent with Client-Side Tools

In this guide, we'll show you the steps to build a customer service chatbot using Claude 3 and various client-side instruments. This chatbot will have the functionality to search for customer data, pull up details of orders, and even cancel orders for the customer. We will outline the required tools and create mock responses to illustrate what the chatbot can do.


## Set up the enviroment


In [ ]:
%pip install anthropic


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.1/869.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')


In [ ]:
import anthropic

client = anthropic.Client()
MODEL_NAME = "claude-3-opus-20240229"

## Define the client-side tools
Next, we'll define the client-side tools that our chatbot will use to assist customers. We'll create three tools: get_customer_info, get_order_details, and cancel_order.

In [ ]:
tools = [
    {
        "name": "get_customer_info",
        "description": "Retrieves customer information based on their customer ID. Returns the customer's name, email, and phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier for the customer."
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "get_order_details",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "cancel_order",
        "description": "Cancels an order based on the provided order ID. Returns a confirmation message if the cancellation is successful.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order to be cancelled."
                }
            },
            "required": ["order_id"]
        }
    }
]

## Simulate synthetic tool responses
Since we don't have real customer data or order information, we'll simulate synthetic responses for our tools. In a real-world scenario, these functions would interact with your actual customer database and order management system.

In [ ]:
def get_customer_info(customer_id):
    # Simulated customer data
    customers = {
        "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
        "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"}
    }
    return customers.get(customer_id, "Customer not found")

def get_order_details(order_id):
    # Simulated order data
    orders = {
        "O1": {"id": "O1", "product": "Widget A", "quantity": 2, "price": 19.99, "status": "Shipped"},
        "O2": {"id": "O2", "product": "Gadget B", "quantity": 1, "price": 49.99, "status": "Processing"}
    }
    return orders.get(order_id, "Order not found")

def cancel_order(order_id):
    # Simulated order cancellation
    if order_id in ["O1", "O2"]:
        return True
    else:
        return False

## Process tool calls and return results
We'll create a function to process the tool calls made by Claude and return the appropriate results.

In [ ]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_customer_info":
        return get_customer_info(tool_input["customer_id"])
    elif tool_name == "get_order_details":
        return get_order_details(tool_input["order_id"])
    elif tool_name == "cancel_order":
        return cancel_order(tool_input["order_id"])

## Interact with the chatbot
Now, let's create a function to interact with the chatbot. We'll send a user message, process any tool calls made by Claude, and return the final response to the user.

In [ ]:
import json

def chatbot_interaction(user_message, MODEL_NAME="claude-3-opus-20240229"):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {"role": "user", "content": user_message}
    ]

    response = client.beta.tools.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        response = client.beta.tools.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            tools=tools,
            messages=messages
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")

    return final_response

##  Test the chatbot
Let's test our customer service chatbot with a few sample queries.

In [ ]:
chatbot_interaction("Can you tell me the email address for customer C1?")



User Message: Can you tell me the email address for customer C1?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text='<thinking>\nTo retrieve a customer\'s email address, the get_customer_info function should be used. This function requires the customer_id parameter.\n\nIn the request, the user specified customer "C1", so the customer_id can be inferred to be "C1".\n\nSince the required customer_id parameter is provided, the get_customer_info function can be called to retrieve the customer\'s email address.\n</thinking>', type='text'), ToolUseBlock(id='toolu_01WmQETncRqahSdqENdaNYMk', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Tool Used: get_customer_info
Tool Input:
{
  "customer_id": "C1"
}

Tool Result:
{
  "name": "John Doe",
  "email": "john@example.com",
  "phone": "123-456-7890"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(text='The email address for customer C1 is john@example.com.', type='text')]

Final Response: The emai

'The email address for customer C1 is john@example.com.'

In [ ]:
chatbot_interaction("What is the status of order O2?")



User Message: What is the status of order O2?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text='<thinking>\nTo retrieve the status of an order, the get_order_details function is most relevant. It requires an order_id parameter.\n\nThe user has provided an order ID in their request - "O2". So I have the necessary parameter to call the function.\n</thinking>', type='text'), ToolUseBlock(id='toolu_01NJmCyWMzWp46HwjibVJ7Rf', input={'order_id': 'O2'}, name='get_order_details', type='tool_use')]

Tool Used: get_order_details
Tool Input:
{
  "order_id": "O2"
}

Tool Result:
{
  "id": "O2",
  "product": "Gadget B",
  "quantity": 1,
  "price": 49.99,
  "status": "Processing"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(text='Based on the order details, the status of order O2 is Processing.', type='text')]

Final Response: Based on the order details, the status of order O2 is Processing.


'Based on the order details, the status of order O2 is Processing.'

In [ ]:
chatbot_interaction("Please cancel order O1 for me.")



User Message: Please cancel order O1 for me.

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text='<thinking>\nThe relevant function to cancel an order is cancel_order. It requires a single parameter:\n- order_id: The ID of the order to cancel. The user provided this directly as "O1".\n\nSince we have the required order_id parameter, we can proceed with calling the cancel_order function.\n</thinking>', type='text'), ToolUseBlock(id='toolu_01YQWVU6pDNEoRnpUtTnVv4Y', input={'order_id': 'O1'}, name='cancel_order', type='tool_use')]

Tool Used: cancel_order
Tool Input:
{
  "order_id": "O1"
}

Tool Result:
true

Response:
Stop Reason: end_turn
Content: [TextBlock(text='Your order O1 has been successfully cancelled.', type='text')]

Final Response: Your order O1 has been successfully cancelled.


'Your order O1 has been successfully cancelled.'

In [ ]:
chatbot_interaction("Write a python program that prints numbers 0 to 100.")



User Message: Write a python program that prints numbers 0 to 100.

Initial Response:
Stop Reason: end_turn
Content: [TextBlock(text="<thinking>\nThe user's request is to write a Python program that prints the numbers 0 to 100. This is a straightforward programming task that does not require any of the provided tools, which are related to retrieving customer information, order details, and cancelling orders.\n\nTo complete this request, I simply need to write a Python script that uses a loop to print the numbers 0 to 100. No additional information is needed from the user.\n</thinking>\n\nHere is a Python program that prints the numbers 0 to 100:\n\n```python\nfor i in range(101):\n    print(i)\n```\n\nThis uses a for loop with the range() function to iterate through the numbers 0 to 100 (101 is the stop value which is not included). The print() function is used to output each number on a separate line.\n\nLet me know if you have any other questions!", type='text')]

Final Response: <t

"<thinking>\nThe user's request is to write a Python program that prints the numbers 0 to 100. This is a straightforward programming task that does not require any of the provided tools, which are related to retrieving customer information, order details, and cancelling orders.\n\nTo complete this request, I simply need to write a Python script that uses a loop to print the numbers 0 to 100. No additional information is needed from the user.\n</thinking>\n\nHere is a Python program that prints the numbers 0 to 100:\n\n```python\nfor i in range(101):\n    print(i)\n```\n\nThis uses a for loop with the range() function to iterate through the numbers 0 to 100 (101 is the stop value which is not included). The print() function is used to output each number on a separate line.\n\nLet me know if you have any other questions!"

In [ ]:
chatbot_interaction("Write a python program that prints numbers 0 to 100.", MODEL_NAME="claude-3-haiku-20240307")



User Message: Write a python program that prints numbers 0 to 100.

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text="Here's a simple Python program that prints the numbers from 0 to 100:", type='text'), ToolUseBlock(id='toolu_01GY2WP7z2BLkVCpJD1J2Rxh', input={}, name='print_numbers', type='tool_use')]

Tool Used: print_numbers
Tool Input:
{}

Tool Result:
null

Response:
Stop Reason: end_turn
Content: [TextBlock(text='The program is:\n\n```python\nfor i in range(101):\n    print(i)\n```\n\nThis uses a `for` loop to iterate through the numbers from 0 to 100 (inclusive) and prints each number.', type='text')]

Final Response: The program is:

```python
for i in range(101):
    print(i)
```

This uses a `for` loop to iterate through the numbers from 0 to 100 (inclusive) and prints each number.


'The program is:\n\n```python\nfor i in range(101):\n    print(i)\n```\n\nThis uses a `for` loop to iterate through the numbers from 0 to 100 (inclusive) and prints each number.'

In [ ]:
chatbot_interaction("What is the status of order O2?", MODEL_NAME="claude-3-haiku-20240307")



User Message: What is the status of order O2?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text='Okay, let me check the details of order O2 for you.', type='text'), ToolUseBlock(id='toolu_01GWvMpgNs4F7vHwaFjDzgZB', input={'order_id': 'O2'}, name='get_order_details', type='tool_use')]

Tool Used: get_order_details
Tool Input:
{
  "order_id": "O2"
}

Tool Result:
{
  "id": "O2",
  "product": "Gadget B",
  "quantity": 1,
  "price": 49.99,
  "status": "Processing"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(text="Based on the details returned, the status of order O2 is 'Processing'.", type='text')]

Final Response: Based on the details returned, the status of order O2 is 'Processing'.


"Based on the details returned, the status of order O2 is 'Processing'."